In [0]:
dbutils.library.restartPython() 

In [0]:
import importlib
import SegScore as ss
importlib.reload(ss)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs, current_date, dayofmonth
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

from pyspark import StorageLevel
import numpy as np
import os
import seaborn as sns
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [0]:
universo = spark.sql("""
SELECT *
FROM catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_scorecomportamiento_vu.hm_scorevehbehaviorclienterbm A
WHERE codmes in (202502, 202505, 202506)

""")

In [0]:
universo.display()

In [0]:
#generación de parquet
##ruta=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/bcp-edv-fabseg/seg_bhv/VEHICULAR'
#universo.write.format("parquet").partitionBy("CODMES").mode("overwrite").save(ruta)

#https://adlscu1lhclbackp05.blob.core.windows.net/bcp-edv-fabseg/seg_bhv/VEHICULAR/

ruta=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/sdp/01_PREMORA/hm_scorevehbehaviorclienterbm_v2'
universo.coalesce(1).write.format("parquet").mode("overwrite").save(ruta)
 

In [0]:
hoy = date.today()
codmes_ejecucion    =   int((hoy - relativedelta(months=1)).strftime("%Y%m"))
codmes_rd           =   int((hoy - relativedelta(months=13)).strftime("%Y%m"))

print(f'CODMES EJECUCIÓN: {codmes_ejecucion}')
print(f'CODMES RD: {codmes_rd}')

In [0]:
# codmes_ejecucion    =   202505
# codmes_rd           =   202405

# 0. Genera Base Seguimiento

###Tabla productiva: 
catalog_lhcl_prod_bcp.bcp_ddv_adrmmgr_modelpersonas_vu.hm_calibradoprovisioncapitaleconomicobhvtcmod20829 
<br><br>
C.ext_flg_disefs,C.ext_flg_bts,C.ext_tipo_empleo_f

In [0]:
#  validación universo
universo_tc = spark.sql("""
SELECT A.*,B.EAD
FROM catalog_lhcl_prod_bcp.bcp_ddv_adrmmgr_seginfobcaper_vu.mm_seguimientoportafoliodefaulttc A
LEFT JOIN     (
SELECT E.EAD,E.CODCLAVEOPECTA,E.CODMES, M.CODCLAVECTA FROM 
(SELECT A.CODMES, A.EAD, A.CODCLAVEOPECTA FROM catalog_lhcl_prod_bcp.bcp_edv_fabseg.ead_ifrs9_bhv_tc_202505 A
UNION ALL 
SELECT B.CODMES, B.EAD, B.CODCLAVEOPECTA FROM catalog_lhcl_prod_bcp.bcp_edv_fabseg.ead_ifrs9_bhv_tc B WHERE CODMES>=202505) E
LEFT JOIN 
     (
        -- catalog_lhcl_prod_bcp.bcp_edv_fabseg.oracle_relcuenta_vp 
         SELECT DISTINCT N.CODCLAVEOPECTA, N.CODCLAVECTA FROM (
        SELECT DISTINCT CODCLAVEOPECTA, CODCLAVECTA FROM  catalog_lhcl_prod_bcp.bcp_edv_sdp.oracle_relcuenta_vp 
        UNION ALL
        SELECT DISTINCT CODCLAVEOPECTA, CODCLAVECTA FROM  catalog_lhcl_prod_bcp.bcp_edv_sdp.oracle_relcuenta_als
        UNION ALL
        SELECT DISTINCT CODCLAVEOPECTA, CODCLAVECTA  FROM catalog_lhcl_prod_bcp.bcp_edv_fabseg.rel_tarjeta_vp
                                                              ) N
    ) M
ON E.CODCLAVEOPECTA=M.CODCLAVEOPECTA 
             ) B
ON A.CODCLAVECTA=B.CODCLAVECTA AND A.CODMES=B.CODMES
""")


In [0]:
# #  validación universo
# universo_tc = spark.sql("""
# SELECT A.*,B.EAD, C.pd_bhv_tc_4q24_f, C.ext_xb_cal_nue_beh,C.mto_prm_act_u6m_cap_log,C.prod_mto_sld_aho_min_u6m_a3,
#  C.ext_flg_titulos_st,C.ext_pd_cliente,C.tag_tc_2_3,C.FLAG_AFLUENTE,C.FLAG_DORM_2_3,C.FLAG_REV_2_3,C.FLAG_TRAN_2_3
#  --,C.ext_flg_disefs,C.ext_flg_bts,C.ext_tipo_empleo_f
# FROM catalog_lhcl_prod_bcp.bcp_ddv_adrmmgr_seginfobcaper_vu.mm_seguimientoportafoliodefaulttc A



In [0]:
universo_tc.groupBy('CODMES').count().display()

In [0]:
#cargando el campo bloqueo
ruta=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/sdp/01_PREMORA/REPRO_BHV_TC'
bloqueo=spark.read.format("delta").load(ruta) 

In [0]:
# bloqueo.groupBy('flgdef60bloqueoatraso_al','CODMES').count().display()
bloqueo.groupBy('CODMES').count().display()

In [0]:

 #DESTIPSEGMENTOINGRESO2000MORARBM:Tipo de segmento por ingresos de cada cliente que tiene un producto en el BCP
 #FLGDESEMPLEODEPENDIENTEAJUSTEPDH2U3M:Flag Descripcion Empleo Dependiente Ajuste Pago de Haberes 2 Ultimo 3 Mes    
 #FLGMTOLINEACREDITOUTILIZADASOLMAY2000: Flag que indica si el monto promedio de la deuda de las tarjetas de crédito BCP del cliente en los ultimos 3 meses (mtoprincipals_pm3) es mayor a 2000. (1 si es mayor a 2000; 0 caso contrario).
#--desrangoratiolineacreditoutilizadatcbcp :

#--RCC_TIP_CLAS_RIE_SBS_MAX_U6M:  RCC Tipo Clasificacion Riesgo SBS Maximo Ultimo 6 Mes
#--FLGDEUDAHIPOTECARIASBSCLI:Flag que indica si tiene deuda hipotecaria en el sistema financiero
#--FLGPDHCTD2U3M : Flag Pago de Haberes Cantidad 2 Ultimo 3 Mes
#--FLGMTOBTSOLMAYOR0:Flag que indica si el monto balance transfer es mayor a 0 se le asigna el valor de 1, en caso contrario el valor de 0. 
#FLGMTODISEFSOL : Flag que indica si se cuenta con disposición de efectivo. Toma el valor de 1 en caso sea MTODISEFSOL mayor a 0

troncal = spark.sql("""    
                                SELECT CODCLAVEPARTYCLI,CODMES,
                                CASE WHEN TIPSEGMENTOINGRESO2000MORARBM=1 THEN 'AlDia_IngAltos' 
                                WHEN TIPSEGMENTOINGRESO2000MORARBM=2 THEN 'Moroso_IngAltos'
                                WHEN TIPSEGMENTOINGRESO2000MORARBM=3 THEN 'AlDia_IngBajos'
                                WHEN TIPSEGMENTOINGRESO2000MORARBM=4 THEN 'Morosos_IngBajos'
                                END AS FLG_ATRASO,
                                CASE WHEN FLGDESEMPLEODEPENDIENTEAJUSTEPDH2U3M=1 THEN 'Dependiente' ELSE 'No_dependiente' END AS FLG_DEPENDIENTE,
                                numpdbhvclitcv9calibfinal23q2,FLGDEUDAHIPOTECARIASBSCLI,FLGCEF,FLGMTOLINEACREDITOUTILIZADASOLMAY2000,RCC_TIP_CLAS_RIE_SBS_MAX_U6M,FLGPDHCTD2U3M,FLGMTOBTSOLMAYOR0, FLGMTODISEFSOL 
                                FROM catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_scorecomportamiento_vu.HM_SCORETCBEHAVIORCLIENTERBM                                   
                            """)

marca_vulnerable = spark.sql("""    
                                SELECT DISTINCT CODMES, date_format(add_months(to_date(CAST(CODMES AS STRING) || "01", "yyyyMMdd"), 1), "yyyyMM") AS CODMES_USO, CODCLAVEPARTYCLI, DESCLASIFICACIONCATEGORIAMARCAVULNERABLERBM AS MARCA_VULNERABLE 
                                FROM catalog_lhcl_prod_bcp.bcp_ddv_adrmmgr_modelpersonas_vu.hm_portafoliovulnerablemod20717rbm WHERE CODCLAVEPARTYCLI IS NOT NULL
                            """) 



pd_bhv_variables = spark.sql("""SELECT M.codclavepartycli,M.codmes, M.numpd AS pd_bhv_tc_4q24_f,
M.colvariablesmodeloanalitico.xb_cal_nue_beh AS xb_cal_nue_beh, M.colvariablesmodeloanalitico.prod_mto_sld_prm_act_med_u6m AS prod_mto_sld_prm_act_med_u6m,
M.colvariablesmodeloanalitico.prod_mto_sld_aho_min_u6m AS prod_mto_sld_aho_min_u6m, M.colvariablesmodeloanalitico.flg_titulos AS flg_titulos,
M.colvariablesmodeloanalitico.pd_cliente_r2 AS pd_cliente_r2, 
CASE WHEN M.colvariablesmodeloanalitico.codsubsegmento IN ('X1N', 'P1N', 'LEN') THEN 'Afluente' ELSE 'Consumo' END SEGMENTO_BANCA_GF,
M.colvariablesmodeloanalitico.tot_flag_dorm_u3m AS tot_flag_dorm_u3m, M.colvariablesmodeloanalitico.tot_flag_rev_u3m AS tot_flag_rev_u3m, M.colvariablesmodeloanalitico.tot_flag_tran_u3m AS tot_flag_tran_u3m
FROM catalog_lhcl_prod_bcp.bcp_ddv_adrmmgr_modelpersonas_vu.hm_calibradoprovisioncapitaleconomicobhvtcmod20829 M
""")
# apalancamiento = spark.sql("""    
#                                 SELECT A.codmes, A.CODCLAVECUC,A.codclaveunicocli,A.INGRESO_0,A.CUOTA_BCP_TOTAL_0,A.CUOTA_RCC_TOTAL_0    
#                                 FROM catalog_lhcl_prod_bcp.bcp_edv_mmgr.hm_nuevocem_cons_tarj A
#                                 WHERE A.codclavecuc>0
#                             """) 
                        


In [0]:
#Producto BT Y ET
producto = spark.sql(""" SELECT DISTINCT codclavepartycli, producto, codcosecha FROM (
                    SELECT codclavepartycli,producto,codcosecha,ROW_NUMBER() OVER (PARTITION BY codclavepartycli ORDER BY codcosecha DESC) AS rn
FROM catalog_lhcl_prod_bcp.bcp_edv_rbmper_002_v.hm_ventas_tc) Z
WHERE Z.rn=1
                            """) 

In [0]:
base_universo_tc = (
    universo_tc.alias("A")
    .join(pd_bhv_variables.alias("B"),(col("A.CODMES")==col("B.codmes")) & (col("A.CODCLAVEPARTYCLI")==col("B.codclavepartycli")),"left" )
    .join(troncal.alias("F"),  (col("A.CODMES") == col("F.CODMES")) & (col("A.codclavepartycli") == col("F.codclavepartycli")), "left" )
    .join(marca_vulnerable.alias("E"), (col("A.CODMES") == col("E.CODMES")) & (col("A.CODCLAVEPARTYCLI") == col("E.CODCLAVEPARTYCLI")), "left")
    .join(bloqueo.alias("G"), (col("A.CODMES") == col("G.CODMES")) & (col("A.CODCLAVECTA") == col("G.CODCLAVECTA")), "left")
    .join(producto.alias("P"), (col("A.CODCLAVEPARTYCLI") == col("P.codclavepartycli")), "left")
   #Filtro de campos
    .filter(col("B.pd_bhv_tc_4q24_f").isNotNull()  & (col("G.flgdef60bloqueoatraso_al") == 0 ) #& col("A.EAD").isNotNull() 
            ) 

    .select(
        col("A.CODMES"), col("A.codclavepartycli"), col("A.codclavecta"), # col("A.codproducto"), 
        col("A.ctdmesmaduracion"),
        col("A.flgdef60bloqueoatraso"), 
        col("A.EAD"),
         
        # when(col("A.EAD").isNull(), 0).otherwise(col("A.EAD")).alias("MTOPRINCIPAL_SOLES"),


        #PD - VARIABLES DEL CALIBRADO
        col("B.pd_bhv_tc_4q24_f"),col("F.numpdbhvclitcv9calibfinal23q2"),
        col("B.xb_cal_nue_beh"),col("B.prod_mto_sld_prm_act_med_u6m"),col("B.prod_mto_sld_aho_min_u6m"),col("B.flg_titulos"),
        col("B.pd_cliente_r2"),
        col("B.tot_flag_dorm_u3m"),col("B.tot_flag_rev_u3m"),col("B.tot_flag_tran_u3m"),
        col("B.SEGMENTO_BANCA_GF"),


        # TARGETS
        col("A.flgdef60bloqueoatrasou12").cast(IntegerType()).alias("DEF12"),
        col("A.flgdef60bloqueoatrasou11").cast(IntegerType()).alias("DEF11"),
        col("A.flgdef60bloqueoatrasou10").cast(IntegerType()).alias("DEF10"),
        col("A.flgdef60bloqueoatrasou9m").cast(IntegerType()).alias("DEF9"),
        col("A.flgdef60bloqueoatrasou8m").cast(IntegerType()).alias("DEF8"),
        col("A.flgdef60bloqueoatrasou7m").cast(IntegerType()).alias("DEF7"),
        col("A.flgdef60bloqueoatrasou6m").cast(IntegerType()).alias("DEF6"),
        col("A.flgdef60bloqueoatrasou5m").cast(IntegerType()).alias("DEF5"),
        col("A.flgdef60bloqueoatrasou4m").cast(IntegerType()).alias("DEF4"),
        col("A.flgdef60bloqueoatrasou3m").cast(IntegerType()).alias("DEF3"),

      
        ## DRIVERS
        #when(col("A.FLAG_AFLUENTE")==1, 'Afluente').otherwise('Consumo').alias("SEGMENTO_BANCA_GF"),
        col("F.FLG_ATRASO"),
        col("F.FLG_DEPENDIENTE"),
        col("F.FLGDEUDAHIPOTECARIASBSCLI"), 
        col("F.FLGCEF"),col("F.FLGMTOLINEACREDITOUTILIZADASOLMAY2000"),col("F.RCC_TIP_CLAS_RIE_SBS_MAX_U6M"), 
        col("F.FLGPDHCTD2U3M"),col("F.FLGMTOBTSOLMAYOR0"),col("F.FLGMTODISEFSOL"),
        col("P.producto"),
        col("A.desrangolineacreditoutilizadatcu3m").alias("RANGO_LINEA_ULT3M"),
        #MARCA VULNERABLE

        when(col("E.MARCA_VULNERABLE") == '3. No Vulnera', '3. No Vulnerable')
        .when(col("E.MARCA_VULNERABLE")  == '2. En observa', '2. En Obs')
        .when(col("E.MARCA_VULNERABLE")  == '1. Vulnerable', '1. Vulnerable Critico')
        .otherwise('3. No Vulnerable').alias("MARCA_VULNERABLE"),
        ################## MTOINGRESO_FINAL #######################
        when(col("A.EAD").between(0, 1999), '1.<0 - 2k>')
        .when(col("A.EAD").between(2000, 4999), '2.<2 - 5k>')
        .when(col("A.EAD").between(5000, 9999), '3.<5 - 10k>')
        .otherwise('4.<10k - mas>').alias("RANGO_INGRESOS"),
        ################### CODCOSECHA #############################
        col("A.CODCOSECHA").alias("CODCOSECHA"),
        when(col("A.codcosecha") <= 202205, '1. Menor 202205 ')
        .when(col("A.codcosecha").between(202206, 202306), '2. 202206-202306')
        .when(col("A.codcosecha").between(202307, 202404), '3. 202307-202404')
        .when(col("A.codcosecha") >= 202501, '4.Mayor 202501')
        .otherwise('99. missing').alias("RANGO_CODCOSECHA")
    )
)  

In [0]:
rds = base_universo_tc.groupBy(
    "CODMES"
).agg(

    F.sum(lit(1))                                                                                                                       .alias("N"),
    F.sum( col("DEF12") )                                                                                                               .alias("DEF_CERRADO_XN"),
    F.sum( col("EAD") ) .alias("M") ,                                                                               
    F.sum( col("DEF12") * col("EAD") )                                                                                    .alias("DEF_CERRADO_XM"),
    (F.sum( col("DEF12") )  / F.sum(lit(1)))  .alias("RD_CUENTAS"),
    ( F.sum( col("DEF12") * col("EAD") )  / F.sum( col("EAD") )  )  .alias("RD_MONTOS"),
) 

In [0]:
rds.display()

In [0]:
df = base_universo_tc.withColumn(
    "TIPO_CSEGPRO",
    when(
        (col("SEGMENTO_BANCA_GF") == "Afluente") & (col("tot_flag_rev_u3m") != 0) , "AF_REV"
        ).when(
        (col("SEGMENTO_BANCA_GF") == "Afluente") & (col("tot_flag_tran_u3m") != 0) , "AF_TRAN"
        ).when(
        (col("SEGMENTO_BANCA_GF") == "Afluente") & (col("tot_flag_dorm_u3m") != 0) , "AF_DOR"
).when(
        (col("SEGMENTO_BANCA_GF") == "Consumo") & (col("tot_flag_rev_u3m") != 0) , "CO_REV"
        ).when(
        (col("SEGMENTO_BANCA_GF") == "Consumo") & (col("tot_flag_tran_u3m") != 0) , "CO_TRAN"
        ).when(
        (col("SEGMENTO_BANCA_GF") == "Consumo") & (col("tot_flag_dorm_u3m") != 0) , "CO_DOR"
).otherwise("OTROS")
).withColumn("MORA_30_3", col("DEF3") 
).withColumn(
    "UTILI_LINEA",
when((col("SEGMENTO_BANCA_GF") == "Afluente") & (col("RANGO_LINEA_ULT3M") == "5. >75%") , "1. >75%-Af")
.when((col("SEGMENTO_BANCA_GF") == "Afluente") & (col("RANGO_LINEA_ULT3M").isin("2. <=25%","3. <=50%","1.SALDO_PROM_3M <=100","4. <=75%") ) , "1. <=75%-Af")
.when((col("SEGMENTO_BANCA_GF") == "Consumo") & (col("RANGO_LINEA_ULT3M") == "5. >75%") , "1. >75%-Con")
.when((col("SEGMENTO_BANCA_GF") == "Consumo") & (col("RANGO_LINEA_ULT3M").isin("2. <=25%","3. <=50%","1.SALDO_PROM_3M <=100","4. <=75%") ) , "1. <=75%-Con")
.otherwise("OTROS") 
)

## GENERANDO BASE DE SEGUIMIENTO FINAL
df = df.filter( ( col("ctdmesmaduracion") > 0) & (col("PD_BHV_TC_4Q24_F").isNotNull()) )

In [0]:
df.groupBy('CODMES').count().display()

In [0]:
ruta=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/seg_bhv/TC/BASE_SEGUIMIENTO_BHV_TC_{codmes_ejecucion}'
 # df.write.format("delta").partitionBy("CODMES").mode("overwrite").save(ruta)    

In [0]:
#leer la data guardada
ruta=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/seg_bhv/TC/BASE_SEGUIMIENTO_BHV_TC_{codmes_ejecucion}'
df=spark.read.format("delta").load(ruta)

In [0]:
df.groupBy('CODMES').sum('EAD').display()

In [0]:
df.display()

In [0]:
rds = df.groupBy(
    "CODMES"
).agg(

    F.sum(lit(1))                                                                                                          .alias("N"),
    F.sum( col("DEF12") )                                                                                                  .alias("DEF_CERRADO_XN"),
    F.sum( col("EAD") ) .alias("M") ,                                                                               
    F.sum( col("DEF12") * col("EAD") )                                                                                     .alias("DEF_CERRADO_XM"),
    (F.sum( col("DEF12") )  / F.sum(lit(1)))  .alias("RD_CUENTAS"),
    ( F.sum( col("DEF12") * col("EAD") )  / F.sum( col("EAD") )  )  .alias("RD_MONTOS"),
) 

In [0]:
rds.display()

In [0]:
#df.display()

In [0]:
#df.groupBy('CODMES').count().display()

In [0]:
#df.display()

# 1. Seguimiento General

In [0]:
", ".join(df.columns) 

In [0]:
df.groupBy("CODMES").count().orderBy("CODMES").tail(40) 
# [Row(CODMES=202309, count=1233732),
#  Row(CODMES=202310, count=1216054),
#  Row(CODMES=202311, count=1239360),
#  Row(CODMES=202312, count=1244568),
#  Row(CODMES=202401, count=1245609),
#  Row(CODMES=202402, count=1254253),
#  Row(CODMES=202403, count=1260872),
#  Row(CODMES=202404, count=1258319),
#  Row(CODMES=202405, count=1276928),
#  Row(CODMES=202406, count=1288450),
#  Row(CODMES=202407, count=1294888),
#  Row(CODMES=202408, count=1306650),
#  Row(CODMES=202409, count=1273386),
#  Row(CODMES=202410, count=1327779),
#  Row(CODMES=202411, count=1329302),
#  Row(CODMES=202412, count=1347429),
#  Row(CODMES=202501, count=1354551),
#  Row(CODMES=202502, count=1360560),
#  Row(CODMES=202503, count=1373498),
#  Row(CODMES=202504, count=1380515)]


In [0]:
# Objeto de Seguimiento

objSeg = ss.MonitorScore_v01(
  mes_seguimiento       = str(codmes_ejecucion),
  base_monitoreo        = df,
  pd1                   = 'PD_BHV_TC_4Q24_F',
  pd2                   = 'numpdbhvclitcv9calibfinal23q2',
  monto_credito         = 'EAD',
  query_univ            = 'CODMES > 202301', #DEF_60_BLOQ_REF_0 = 0 AND CODMES > 202302
  bandas                = 'Vasiseck', #Vasiseck
  codmes_default        = codmes_rd,
  meses_ventana_target  = 12,
  meses_atraso_target   = 2
) 

In [0]:
bhv_tc, cubo_c_10, cubo_m_10, factor_c_10, factor_m_10 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "PRODUCTO = 'PTC'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'Best_proy', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.15,
  eje_y_xm           = 0.15,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
cubo_m_10

In [0]:
bhv_tc, cubo_c_7, cubo_m_7, factor_c_7, factor_m_7 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "UTILI_LINEA = '1. >75%-Con'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'Best_proy', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.35,
  eje_y_xm           = 0.35,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
bhv_tc, cubo_c_8, cubo_m_8, factor_c_8, factor_m_8 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "UTILI_LINEA = '1. <=75%-Con'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'Best_proy', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.15,
  eje_y_xm           = 0.15,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
bhv_tc, cubo_c_9, cubo_m_9, factor_c_9, factor_m_9 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "UTILI_LINEA = '1. <=75%-Af'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'Best_proy', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.15,
  eje_y_xm           = 0.15,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
bhv_tc, cubo_c_10, cubo_m_10, factor_c_10, factor_m_10 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "UTILI_LINEA = '1. >75%-Af'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'Best_proy', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.25,
  eje_y_xm           = 0.25,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
cubo_m_10

In [0]:


#cubo_m_7.where("codmes > 202306").display()

cubo_m_7[cubo_m_7.CODMES > 202306]



In [0]:
# bhv_tc, cubo_c_8, cubo_m_8, factor_c_8, factor_m_8 = objSeg.EvolCal(
#   # Configuración de filtro de uso y pd de calibración
#   query_uso          = "UTILI_LINEA = '1. <=75%-Con'",
#   pd_seg             = 'PD_BHV_TC_4Q24_F',
#   pd_comparacion     = True,
#   proys_def_temp     = 6,
#   proys_mor_temp     = 3,
#   rd_aux             = 'RD6',
#   rd_aux2            = 'MORA_30_3',
#   mora_temp          = 'MORA_30_3',
#   amplitud           = 1,
#   moviles            = False,

#   # Configuración de proyecciones por cuentas
#   proy_def_temp_xc   = 'Best_proy', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
#   suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
#   proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
#   suav_mor_temp_xc   = 1,
#   prof_hist_xc       = 14, # 

#   # Configuración de proyecciones por montos
#   proy_def_temp_xm   = 'Best_proy', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
#   suav_def_temp_xm   = 1,
#   proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
#   suav_mor_temp_xm   = 1,
#   prof_hist_xm       = 14,

#   # Detalles del gráfico de calibración
#   titulo             = 'Calibración Bhv TC',
#   pos_leyenda        = (0.5, -0.25),
#   eje_y_xc           = 0.35,
#   eje_y_xm           = 0.35,
#   dim_grafico        = (25, 6.5),
#   punt_mora          = 100,
#   width_bar          = 1,
#   etiquetas          = True,
#   pos_etiquetas_xc   = 10,
#   pos_etiquetas_xm   = 10,
#   tamaño_etiqueta    = 15,
#   mat_porc           = -1,

#   # Exportar factores para vista driver
#   exportar_factores  = True
# ) 

## Calibración: proyectando para cada segmento establecido

In [0]:
bhv_tc, cubo_c_1, cubo_m_1, factor_c_1, factor_m_1 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "TIPO_CSEGPRO = 'AF_REV'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'Best_proy', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.15,
  eje_y_xm           = 0.15,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
bhv_tc, cubo_c_2, cubo_m_2, factor_c_2, factor_m_2 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "TIPO_CSEGPRO = 'AF_TRAN'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'Best_proy', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'Best_proy', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.10,
  eje_y_xm           = 0.10,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
factor_m_2 

In [0]:
# factor_m_2 

In [0]:

bhv_tc, cubo_c_3, cubo_m_3, factor_c_3, factor_m_3 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "TIPO_CSEGPRO = 'AF_DOR'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met3_xc', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'Best_proy', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.050,
  eje_y_xm           = 0.050,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
factor_m_3

In [0]:
# factor_m_3

In [0]:
bhv_tc, cubo_c_4, cubo_m_4, factor_c_4, factor_m_4 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "TIPO_CSEGPRO = 'CO_TRAN'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   ="np.where(calxcuentas['CODMES'].isin([202407,202408,202409,202410,202411,202411,202412,202501]), met1_xc*1.0, met3_xc)",#"np.where(calxcuentas['CODMES'].isin([202407]), met3_xc*1.3, met3_xc)", #"np.where(calxcuentas['CODMES'].isin([202407]), met3_xc*1.3, met3_xc)", #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(calxmontos['CODMES'].isin([202407,202408,202409,202410,202411,202411,202412,202501]),met1_xm*1.0, met3_xm)",#"np.where(calxmontos['CODMES'].isin([202407]), met3_xm*1.5, met3_xm)",#"np.where(calxmontos['CODMES'].isin([202407]), met3_xm*1.4, met3_xm)", #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  #'np.where(calxmontos["CODMES"].isin([202403, 202404, 202405, 202406, 202407, 202408]), met1_xm*1.07, met1_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.050,
  eje_y_xm           = 0.050,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
factor_m_4

In [0]:

bhv_tc, cubo_c_5, cubo_m_5, factor_c_5, factor_m_5 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "TIPO_CSEGPRO = 'CO_REV'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "np.where(calxcuentas['CODMES'].isin([202407,202408,202409,202410,202411,202411,202412,202501]), met3_xc*1, met3_xc)", #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(calxmontos['CODMES'].isin([202407,202408,202409,202410,202411,202411,202412,202501]), met3_xm*1, met3_xm)",#met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.25,
  eje_y_xm           = 0.25,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
factor_c_5.display() 

In [0]:
bhv_tc, cubo_c_6, cubo_m_6, factor_c_6, factor_m_6 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "TIPO_CSEGPRO = 'CO_DOR'",
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met1_xc', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, # 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met1_xm', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.08,
  eje_y_xm           = 0.08,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
factor_c_6 

In [0]:
# bhv_tc, cubo_c_7, cubo_m_7, factor_c_7, factor_m_7 = objSeg.EvolCal(
#   # Configuración de filtro de uso y pd de calibración
#   query_uso          = "SEGMENTO_PF = 'B. Af/Rev/cuota>75%'",
#   pd_seg             = 'PD_BHV_TC_4Q24_F',
#   pd_comparacion     = True,
#   proys_def_temp     = 6,
#   proys_mor_temp     = 3,
#   rd_aux             = 'RD6',
#   rd_aux2            = 'MORA_30_3',
#   mora_temp          = 'MORA_30_3',
#   amplitud           = 1,
#   moviles            = False,

#   # Configuración de proyecciones por cuentas
#   proy_def_temp_xc   = 'met3_xc', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
#   suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
#   proy_mor_temp_xc   = 'det3_xc', #det1_xc, det2_xc, det3_xc, All_proy, best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
#   suav_mor_temp_xc   = 1,
#   prof_hist_xc       = 14, # 

#   # Configuración de proyecciones por montos
#   proy_def_temp_xm   = 'met3_xm', #met1_xm, met2_xm, met3_xm, All_proy, best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
#   suav_def_temp_xm   = 1,
#   proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, best_proy
#   suav_mor_temp_xm   = 1,
#   prof_hist_xm       = 14,

#   # Detalles del gráfico de calibración
#   titulo             = 'Calibración Bhv TC',
#   pos_leyenda        = (0.5, -0.25),
#   eje_y_xc           = 0.25,
#   eje_y_xm           = 0.25,
#   dim_grafico        = (25, 6.5),
#   punt_mora          = 100,
#   width_bar          = 1,
#   etiquetas          = True,
#   pos_etiquetas_xc   = 10,
#   pos_etiquetas_xm   = 10,
#   tamaño_etiqueta    = 15,
#   mat_porc           = -1,

#   # Exportar factores para vista driver
#   exportar_factores  = True
# ) 

In [0]:
#cubo_m_7

## Suma Producto por segmentos (Ponderación)

In [0]:
bhv_tc, cubo_c_co, cubo_m_co, factor_agg_c_1, factor_agg_m_1  = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = 'SEGMENTO_BANCA_GF = "Consumo"', # "TIPO_CSEGPRO = 'REG_AF_LD'"
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met1_xc', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', # det1_xc, det2_xc, det3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, 
  fact_to_proy_xc    = [factor_c_4, factor_c_5, factor_c_6],

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met1_xm', #met1_xm, met2_xm, met3_xm, All_proy, Best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, Best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  fact_to_proy_xm    = [factor_m_4, factor_m_5, factor_m_6],

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.15,
  eje_y_xm           = 0.15,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
cubo_m_co.display()

In [0]:
bhv_tc, cubo_c_af, cubo_m_af, factor_agg_c_2, factor_agg_m_2  = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = 'SEGMENTO_BANCA_GF = "Afluente"', # "TIPO_CSEGPRO = 'REG_AF_LD'"
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met1_xc', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', # det1_xc, det2_xc, det3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, 
  fact_to_proy_xc    = [factor_c_1, factor_c_2, factor_c_3],

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met1_xm', #met1_xm, met2_xm, met3_xm, All_proy, Best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, Best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  fact_to_proy_xm    = [factor_m_1, factor_m_2, factor_m_3],

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.10,
  eje_y_xm           = 0.10,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:

cbhv_tc, cubo_c, cubo_m, factor_agg_c_3, factor_agg_m_3 = objSeg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = '1=1', 
  pd_seg             = 'PD_BHV_TC_4Q24_F',
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 3,
  rd_aux             = 'RD6',
  rd_aux2            = 'MORA_30_3',
  mora_temp          = 'MORA_30_3',
  amplitud           = 1,
  moviles            = False,

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'met1_xc', #met1_xc, met2_xc, met3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (met1_xc + met2_xc)/ 2 etc.
  suav_def_temp_xc   = 1, # factor suavizamiento para def temprano
  proy_mor_temp_xc   = 'det3_xc', # det1_xc, det2_xc, det3_xc, All_proy, Best_proy --Ademas puede operar con las metodologías: (det1_xc + det2_xc)/ 2 etc.
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14, 
  fact_to_proy_xc    = [factor_agg_c_1, factor_agg_c_2], 

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = 'met1_xm', #met1_xm, met2_xm, met3_xm, All_proy, Best_proy , 'np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)'
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det3_xm', #det1_xm, det2_xm, det3_xm, All_proy, Best_proy
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  fact_to_proy_xm    = [factor_agg_m_1, factor_agg_m_2],

  # Detalles del gráfico de calibración
  titulo             = 'Calibración Bhv TC',
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.10,
  eje_y_xm           = 0.10,
  dim_grafico        = (25, 6.5),
  punt_mora          = 100,
  width_bar          = 1,
  etiquetas          = True,
  pos_etiquetas_xc   = 10,
  pos_etiquetas_xm   = 10,
  tamaño_etiqueta    = 15,
  mat_porc           = -1,

  # Exportar factores para vista driver
  exportar_factores  = True
) 

In [0]:
cubo_m

## Vista Cubo Power BI 

In [0]:
def procesar_factores(n_SegProy, Mes_Def):
    df_factores = pd.DataFrame()  # Inicializar un DataFrame vacío
    
    for i in range(1, n_SegProy + 1):
        # Generar los nombres de los DataFrames dinámicamente
        fc = globals()[f'factor_c_{i}']
        fm = globals()[f'factor_m_{i}']
        
        # Añadir la columna SEG_PROYECCION
        fc['SEG_PROYECCION'] = i
        fc['CODMES_DEF'] = Mes_Def

        u3m_c = fc['CODMES'].nlargest(3)
        u3m_m = fm['CODMES'].nlargest(3)

        # Rellenar los valores nulos en la columna RD_PROY_DEF con los valores de la columna DET_PROY_DEF para los últimos tres meses
        fc.loc[fc['CODMES'].isin(u3m_c), 'RD_PROY_DEF'] = fc['DET_PROY_DEF']
        fm.loc[fc['CODMES'].isin(u3m_m), 'RD_PROY_DEF'] = fm['DET_PROY_DEF']

        # Realizar la unión de los DataFrames
        df_resultante = pd.merge(fc, fm, on="CODMES", how="inner", suffixes=('_fc', '_fm'))

        print(df_resultante[['CODMES', 'RD_PROY_DEF_fm', 'FAC_CAL_DRI_M']].head())

        # Seleccionar las columnas deseadas
        df_resultante = df_resultante[['CODMES_DEF','CODMES', 'SEG_PROYECCION', 'RD_PROY_DEF_fm', 'FAC_CAL_DRI_M', 'RD_PROY_DEF_fc', 'FAC_CAL_DRI_N']]

        # Reemplazar los valores de factores NaN por 0 y RD por 0.005
        df_resultante[['FAC_CAL_DRI_M', 'FAC_CAL_DRI_N']] = df_resultante[['FAC_CAL_DRI_M', 'FAC_CAL_DRI_N']].fillna(0)
        df_resultante[['RD_PROY_DEF_fm', 'RD_PROY_DEF_fc']] = df_resultante[['RD_PROY_DEF_fm', 'RD_PROY_DEF_fc']].fillna(0.005)

        # Cambiar los nombres de las columnas
        df_resultante.rename(columns={'RD_PROY_DEF_fm': 'proyeccion_montos'}, inplace=True)
        df_resultante.rename(columns={'FAC_CAL_DRI_M': 'factor_montos'}, inplace=True)
        df_resultante.rename(columns={'RD_PROY_DEF_fc': 'proyeccion_cuentas'}, inplace=True)
        df_resultante.rename(columns={'FAC_CAL_DRI_N': 'factor_cuentas'}, inplace=True)

        # Concatenar el DataFrame resultante al DataFrame final
        df_factores = pd.concat([df_factores, df_resultante], ignore_index=True)

    return df_factores

df_factores = procesar_factores(6, 202505)  # Consolidación de factores y valores

In [0]:
df_factores = spark.createDataFrame(df_factores) # pandas a pyspark df

## Genera Cubo Calibración

In [0]:
ruta=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/seg_bhv/TC/FACTORES_BHV_TC'
df_factores.write.format("delta").partitionBy("CODMES").mode("overwrite").save(ruta) 

In [0]:
ruta=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/seg_bhv/TC/FACTORES_BHV_TC'
factores_bhv_tc=spark.read.format("delta").load(ruta) 
factores_bhv_tc.display()

In [0]:

objSeg.base_monitoreo = objSeg.base_monitoreo.withColumn("SEG_PROYECCION",
                                                                    
                                                                        when(objSeg.base_monitoreo.TIPO_CSEGPRO == "AF_REV", 1).
                                                                        when(objSeg.base_monitoreo.TIPO_CSEGPRO == "AF_TRAN", 2).
                                                                        when(objSeg.base_monitoreo.TIPO_CSEGPRO == "AF_DOR", 3).
                                                                        when(objSeg.base_monitoreo.TIPO_CSEGPRO == "CO_TRAN", 4).
                                                                        when(objSeg.base_monitoreo.TIPO_CSEGPRO == "CO_REV", 5).
                                                                        when(objSeg.base_monitoreo.TIPO_CSEGPRO == "CO_DOR", 6))                                                                        

In [0]:
MES_DEF_ACT = codmes_ejecucion
default = 12

# Función para calcular los meses anteriores
def calculate_previous_months(codmes, cant):
  numeromes = (codmes // 100) * 12 + (codmes % 100)
  return ((numeromes - 1 + cant) // 12) * 100 + ((numeromes + 11 - 11 * cant) % 12) + 1

#Calculando meses default
codmes_a1 = calculate_previous_months(MES_DEF_ACT, -default+11)
codmes_a2 = calculate_previous_months(MES_DEF_ACT, -default+10)
codmes_a3 = calculate_previous_months(MES_DEF_ACT, -default+9)
codmes_a4 = calculate_previous_months(MES_DEF_ACT, -default+8)
codmes_a5 = calculate_previous_months(MES_DEF_ACT, -default+7)
codmes_a6 = calculate_previous_months(MES_DEF_ACT, -default+6)
codmes_a7 = calculate_previous_months(MES_DEF_ACT, -default+5)
codmes_a8 = calculate_previous_months(MES_DEF_ACT, -default+4)
codmes_a9 = calculate_previous_months(MES_DEF_ACT, -default+3)
codmes_a10 = calculate_previous_months(MES_DEF_ACT, -default+2)
codmes_a11 = calculate_previous_months(MES_DEF_ACT, -default+1)
codmes_a12 = calculate_previous_months(MES_DEF_ACT, -default)




In [0]:
df_t = objSeg.base_monitoreo

df_t = df_t.withColumn("ULT_RD", when(col("CODMES") == codmes_a11, col("DEF11"))
                                 .when(col("CODMES") == codmes_a10, col("DEF10"))
                                 .when(col("CODMES") == codmes_a9, col("DEF9"))
                                 .when(col("CODMES") == codmes_a8, col("DEF8"))
                                 .when(col("CODMES") == codmes_a7, col("DEF7"))
                                 .when(col("CODMES") == codmes_a6, col("DEF6"))
                                 .when(col("CODMES") == codmes_a5, col("DEF5"))
                                 .when(col("CODMES") == codmes_a4, col("DEF4"))
                                 .when(col("CODMES") == codmes_a3, col("DEF3"))
                                 .otherwise(0))




df_proy = df_t.join(df_factores, (df_t.CODMES == df_factores.CODMES) & (df_t.SEG_PROYECCION == df_factores.SEG_PROYECCION), "left")  

df_cal01 = df_proy.select(
    df_t["*"],
    df_factores.factor_montos,
    df_factores.factor_cuentas,
    (col("ULT_RD") * df_factores.factor_montos).alias("def12_proy_factor_m"),
    (col("ULT_RD") * df_factores.factor_cuentas).alias("def12_proy_factor_n"),
    df_factores.proyeccion_montos.alias("def12_proy_valor_m"),
    df_factores.proyeccion_cuentas.alias("def12_proy_valor_n") 
) 

In [0]:
# df_factores.filter(df_factores["SEG_PROYECCION"] == 2 ).display() 

In [0]:
## Añadir flag de los 3 últimos meses en base a la RD6 o en base a los últimos 3 meses
df_cal01 = df_cal01.withColumn("FLG_U3M_RD6", when(col("CODMES").between(codmes_a8, codmes_a6), 1).otherwise(0)) \
           .withColumn("FLG_U3M", when(col("CODMES").between(codmes_a4, codmes_a2), 1).otherwise(0)) 

## Agregando Drivers

## Generando cubo final

In [0]:
df_cal01.dtypes 

In [0]:
# df_cal01 = df_cal01.withColumn("PD_BHV_CEF_4Q23_F", col("PD_BHV_CEF_4Q23_F").cast("float"))
# df_cal01 = df_cal01.fillna({'PD_BHV_CEF_4Q23_F': 0}) 

In [0]:
#ruta=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/seg_bhv/TC/df_cal01'
#df_cal01.write.format("delta").partitionBy("CODMES").mode("overwrite").save(ruta) 

In [0]:
df.display()

In [0]:
cubo_principal = df_cal01.groupBy(
    "CODMES",
    "SEG_PROYECCION",
    "FLG_U3M_RD6",
    "FLG_U3M",
    lit("TC")                                                                           .alias("PRODUCTO"),
    col("SEGMENTO_BANCA_GF")                                                            .alias("EJE_P01"),
    col("TIPO_CSEGPRO")                                                                  .alias("EJE_P02"),
    col("PRODUCTO")                                                                      .alias("EJE_P03"),
    col("FLGPDHCTD2U3M")                                                                 .alias("EJE_P04"),
    col("FLGMTOBTSOLMAYOR0")                                                              .alias("EJE_P05"),
    col("MARCA_VULNERABLE")                                                               .alias("EJE_S01"),
    col("FLG_ATRASO")                                                                     .alias("EJE_S02"),
    col("UTILI_LINEA")                                                                   .alias("EJE_S03"),
    col("RANGO_INGRESOS")                                                                  .alias("EJE_S04"),
    col("RANGO_LINEA_ULT3M")                                                               .alias("EJE_S05"),
    col("RCC_TIP_CLAS_RIE_SBS_MAX_U6M")                                                    .alias("EJE_S06"),
    col("FLGMTOLINEACREDITOUTILIZADASOLMAY2000")                                           .alias("EJE_S07"),
    col("FLG_DEPENDIENTE")                                                                 .alias("EJE_S08"),
    col("FLGCEF")                                                                          .alias("EJE_S09"),
    lit("")                                                                      .alias("EJE_S10"),
    lit("")                                                                                                  .alias("EJE_S11"),
    lit("")                                                                                                 .alias("EJE_S12"),
    lit("")                                                                                                 .alias("EJE_S13"),
    lit("")                                                                                                 .alias("EJE_S14"),
    col("RANGO_CODCOSECHA")                                                                                 .alias("EJE_S15")
).agg(

    F.sum(lit(1))                                                                                                                       .alias("N"),
    F.sum( col("PD_BHV_TC_4Q24_F") )                                                                                                   .alias("PD_XN_01"),
    F.sum( col("NUMPDBHVCLITCV9CALIBFINAL23Q2") )                                                                                                   .alias("PD_XN_02"),
    F.sum( lit("") )                                                                                                                    .alias("PD_XN_03"),
    F.sum( col("DEF12") )                                                                                                               .alias("DEF_CERRADO_XN"),
    F.sum( col("DEF6") )                                                                                                                .alias("DEF_TEMPRANO_XN"),
    F.sum( col("DEF3") )                                                                                                                .alias("DEF_MORA_TEMPRANA_XN"),
    F.sum(when(col("CODMES").between(codmes_a11, codmes_a6), col("def12_proy_factor_n")).otherwise(0) )                                 .alias("DEF_PROY_FACTOR_DT_XN"),
    F.sum(when(col("CODMES").between(codmes_a5, codmes_a3), col("def12_proy_factor_n")).otherwise(0) )                                  .alias("DEF_PROY_FACTOR_MT_XN"),
    F.sum(when(col("CODMES").between(codmes_a11, codmes_a6), col("def12_proy_valor_n")).otherwise(0))                                   .alias("DEF_PROY_VALOR_DT_XN"),
    F.sum(when(col("CODMES").between(codmes_a5, codmes_a3), col("def12_proy_valor_n")).otherwise(0))                                    .alias("DEF_PROY_VALOR_MT_XN"),

    F.sum( col("EAD") )                                                                                                   .alias("M"),
    F.sum( col("PD_BHV_TC_4Q24_F") * col("EAD") )                                                                        .alias("PD_XM_01"),
    F.sum( col("NUMPDBHVCLITCV9CALIBFINAL23Q2") * col("EAD") )                                                           .alias("PD_XM_02"),
    F.sum( lit("") )                                                                                                                    .alias("PD_XM_03"),
    F.sum( col("DEF12") * col("EAD") )                                                                                    .alias("DEF_CERRADO_XM"),
    F.sum( col("DEF6") * col("EAD") )                                                                                     .alias("DEF_TEMPRANO_XM"),
    F.sum( col("DEF3") * col("EAD") )                                                                                     .alias("DEF_MORA_TEMPRANA_XM"),
    F.sum(when(col("CODMES").between(codmes_a11, codmes_a6), col("def12_proy_factor_m") * col("EAD") ).otherwise(0) )     .alias("DEF_PROY_FACTOR_DT_XM"),
    F.sum(when(col("CODMES").between(codmes_a5, codmes_a3), col("def12_proy_factor_m")* col("EAD") ).otherwise(0) )       .alias("DEF_PROY_FACTOR_MT_XM"),
    F.sum(when(col("CODMES").between(codmes_a11, codmes_a6), col("def12_proy_valor_m") * col("EAD") ).otherwise(0))       .alias("DEF_PROY_VALOR_DT_XM"),
    F.sum(when(col("CODMES").between(codmes_a5, codmes_a3), col("def12_proy_valor_m") * col("EAD") ).otherwise(0))        .alias("DEF_PROY_VALOR_MT_XM")

) 

In [0]:
cubo_principal.display()

In [0]:
# Colocar en nulos valores que estan fuera de tiempo
cubo_principal = cubo_principal.withColumn("DEF_CERRADO_XN", when(col("CODMES") >= codmes_a11 ,  lit(None) ).otherwise(col("DEF_CERRADO_XN")))
cubo_principal = cubo_principal.withColumn("DEF_CERRADO_XM", when(col("CODMES") >= codmes_a11 ,  lit(None) ).otherwise(col("DEF_CERRADO_XM")))
cubo_principal = cubo_principal.withColumn("DEF_TEMPRANO_XN", when(col("CODMES") >= codmes_a5 ,  lit(None) ).otherwise(col("DEF_TEMPRANO_XN")))
cubo_principal = cubo_principal.withColumn("DEF_TEMPRANO_XM", when(col("CODMES") >= codmes_a5 ,   lit(None) ).otherwise(col("DEF_TEMPRANO_XM")))
cubo_principal = cubo_principal.withColumn("DEF_MORA_TEMPRANA_XN", when(col("CODMES") >= codmes_a2 ,   lit(None) ).otherwise(col("DEF_MORA_TEMPRANA_XN")))
cubo_principal = cubo_principal.withColumn("DEF_MORA_TEMPRANA_XM", when(col("CODMES") >= codmes_a2 ,  lit(None) ).otherwise(col("DEF_MORA_TEMPRANA_XM")))

cubo_principal = cubo_principal.withColumn("DEF_PROY_FACTOR_DT_XN", when( ( col("CODMES") >= codmes_a5) | ( col("CODMES") <=  codmes_a12),  lit(None) ).otherwise(col("DEF_PROY_FACTOR_DT_XN")))
cubo_principal = cubo_principal.withColumn("DEF_PROY_VALOR_DT_XN", when( ( col("CODMES") >= codmes_a5) | ( col("CODMES") <=  codmes_a12),  lit(None) ).otherwise(col("DEF_PROY_VALOR_DT_XN")))
cubo_principal = cubo_principal.withColumn("DEF_PROY_FACTOR_DT_XM", when( ( col("CODMES") >= codmes_a5) | ( col("CODMES") <=  codmes_a12) ,  lit(None) ).otherwise(col("DEF_PROY_FACTOR_DT_XM")))
cubo_principal = cubo_principal.withColumn("DEF_PROY_VALOR_DT_XM", when( ( col("CODMES") >= codmes_a5) |( col("CODMES") <=  codmes_a12) ,  lit(None) ).otherwise(col("DEF_PROY_VALOR_DT_XM")))

cubo_principal = cubo_principal.withColumn("DEF_PROY_FACTOR_MT_XN", when( ( col("CODMES") >= codmes_a2) | ( col("CODMES") <=  codmes_a6),  lit(None) ).otherwise(col("DEF_PROY_FACTOR_MT_XN")))
cubo_principal = cubo_principal.withColumn("DEF_PROY_VALOR_MT_XN", when( ( col("CODMES") >= codmes_a2) | ( col("CODMES") <=  codmes_a6),  lit(None) ).otherwise(col("DEF_PROY_VALOR_MT_XN")))
cubo_principal = cubo_principal.withColumn("DEF_PROY_FACTOR_MT_XM", when( ( col("CODMES") >= codmes_a2) | ( col("CODMES") <=  codmes_a6) ,  lit(None) ).otherwise(col("DEF_PROY_FACTOR_MT_XM")))
cubo_principal = cubo_principal.withColumn("DEF_PROY_VALOR_MT_XM", when( ( col("CODMES") >= codmes_a2) | ( col("CODMES") <=  codmes_a6) ,  lit(None) ).otherwise(col("DEF_PROY_VALOR_MT_XM")))


In [0]:
def df_to_table( df, tableName ):
  
  abfs_url = "abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net"
  finalLocation = abfs_url[:8] + os.path.normpath(f"{abfs_url[8:]}/seg_bhv/DASHBOARDS/{tableName}")
  schema = "catalog_lhcl_prod_bcp.bcp_edv_fabseg"

  dropQuery = f"DROP TABLE IF EXISTS {schema}.{tableName}"
  print(dropQuery)
  spark.sql(dropQuery)
  dbutils.fs.rm(finalLocation, recurse=True)
  
  # Creando Tabla
  df.createOrReplaceTempView(tableName)
  createQuery = f"""
    CREATE TABLE {schema}.{tableName}
    LOCATION '{finalLocation}'
    AS SELECT * FROM {tableName}
  """
  print(createQuery)
  spark.sql(createQuery) 

In [0]:
df_to_table(cubo_principal, "pbi_cubo_behavior_tc")  

In [0]:
%sql
SELECT CODMES, PD_XM_01 FROM catalog_lhcl_prod_bcp.bcp_edv_fabseg.pbi_cubo_behavior_tc
--WHERE CODMES =202505

# 2. Análisis de Cosechas

In [0]:
ruta=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/seg_bhv/TC/df_cal01'
df_cal01.write.format("delta").option("mergeSchema", "true").partitionBy("CODMES").mode("overwrite").save(ruta) 

In [0]:
ruta_df_cal01=f'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/seg_bhv/TC/df_cal01'

df_cal01=spark.read.format("delta").load(ruta_df_cal01)

In [0]:
 df_cal01.display()

In [0]:
cubo_rf = df_cal01.groupBy(
        col("CODMES"),
        lit("TC").alias("MODELO"),
        col("TIPO_CSEGPRO"),
        col("SEGMENTO_BANCA_GF"),
        col("CODCOSECHA").alias("CODCOSECHA"),
        
        when(col("codcosecha") <= 202205, '1. Menor 202205 ')
        .when(col("codcosecha").between(202206, 202306), '2. 202206-202306')
        .when(col("codcosecha").between(202307, 202404), '3. 202307-202404')
        .when(col("codcosecha") >= 202405, '4.Mayor 202405')
        .otherwise('99. missing').alias("RANGO_CODCOSECHA"),


        when((col("codcosecha").between(201901, 201912)), "2019")
        .when((col("codcosecha").between(202001, 202003)), "2020-Q1")
        .when((col("codcosecha").between(202004, 202006)), "2020-Q2")
        .when((col("codcosecha").between(202007, 202009)), "2020-Q3")
        .when((col("codcosecha").between(202010, 202012)), "2020-Q4")
        .when((col("codcosecha").between(202101, 202103)), "2021-Q1")
        .when((col("codcosecha").between(202104, 202106)), "2021-Q2")
        .when((col("codcosecha").between(202107, 202109)), "2021-Q3")
        .when((col("codcosecha").between(202110, 202112)), "2021-Q4")
        .when((col("codcosecha").between(202201, 202203)), "2022-Q1")
        .when((col("codcosecha").between(202204, 202206)), "2022-Q2")
        .when((col("codcosecha").between(202207, 202209)), "2022-Q3")
        .when((col("codcosecha").between(202210, 202212)), "2022-Q4")
        .when((col("codcosecha").between(202301, 202303)), "2023-Q1")
        .when((col("codcosecha").between(202304, 202306)), "2023-Q2")
        .when((col("codcosecha").between(202307, 202309)), "2023-Q3")
        .when((col("codcosecha").between(202310, 202312)), "2023-Q4")
        .when((col("codcosecha").between(202401, 202403)), "2024-Q1")
        .when((col("codcosecha").between(202404, 202406)), "2024-Q2")
        .when((col("codcosecha").between(202407, 202409)), "2024-Q3")
        .when((col("codcosecha").between(202410, 202412)), "2024-Q4")
        .when((col("codcosecha").between(202501, 202503)), "2025-Q1")
        .when((col("codcosecha").between(202504, 202506)), "2025-Q2")
        .otherwise("1. Anterior")
        .alias("CODCOSECHA_Q"),

        when((col("CODMES").between(202301, 202303)), "2023-Q1")
        .when((col("CODMES").between(202304, 202306)), "2023-Q2")
        .when((col("CODMES").between(202307, 202309)), "2023-Q3")
        .when((col("CODMES").between(202310, 202312)), "2023-Q4")
        .when((col("CODMES").between(202401, 202403)), "2024-Q1")
        .when((col("CODMES").between(202404, 202406)), "2024-Q2")
        .when((col("CODMES").between(202407, 202409)), "2024-Q3")
        .when((col("CODMES").between(202410, 202412)), "2024-Q4")
        .when((col("CODMES").between(202501, 202503)), "2025-Q1")
        .when((col("CODMES").between(202504, 202506)), "2025-Q2")
        .otherwise("1. Anterior")
        .alias("CODMES_Q")
        
        ).agg(

    F.sum(lit(1)).alias("N"),
    F.sum( col("EAD") ).alias("M"),

    F.sum( col("DEF12") ).alias("DEF12_XN"),
    F.sum( col("DEF6") ).alias("DEF6_XN"),
     
    F.sum( col("DEF12") * col("EAD") ).alias("DEF12_XM"),
    F.sum( col("DEF6") * col("EAD") ).alias("DEF6_XM")
  
) 

In [0]:
cubo_rf = cubo_rf.withColumn("DEF12_XN", when(col("CODMES") >= codmes_a11 ,  lit(None) ).otherwise(col("DEF12_XN")))
cubo_rf = cubo_rf.withColumn("DEF12_XM", when(col("CODMES") >= codmes_a11 ,  lit(None) ).otherwise(col("DEF12_XM")))
cubo_rf = cubo_rf.withColumn("DEF6_XN", when(col("CODMES") >= codmes_a5 ,  lit(None) ).otherwise(col("DEF6_XN")))
cubo_rf = cubo_rf.withColumn("DEF6_XM", when(col("CODMES") >= codmes_a5 ,  lit(None) ).otherwise(col("DEF6_XM")))

In [0]:
df_to_table(cubo_rf, "pbi_cubo_behavior_tc_rf")  

In [0]:
%sql
SELECT * FROM catalog_lhcl_prod_bcp.bcp_edv_fabseg.pbi_cubo_behavior_tc_rf
